In [3]:
import plotly.express as px
import os
import pandas as pd
import numpy as np
import plotly.graph_objects as go

#import csv
pd.set_option('display.max_rows', None)  # Display all rows

In [4]:
day = "24"
month = "08"
dataset = "ble"

date = day + "_" + month #change date accordingly
date_year = "2023-" + month + "-" + day

folder_path = dataset + '_base/'


def process_file(file_number, date):
    file_path = os.path.join(folder_path, f'{dataset}_{date}_{file_number}.csv')

    df = pd.read_csv(file_path, parse_dates=['ts'])

    # Remove first empty column if it exists
    if df.shape[1] == 4:
        df = df.drop(df.columns[0], axis=1)
    # Remove entries with 'gabarit' equal to 1 or 2
    df_filt = df[(df['id'] == 2) | (df['id'] == 1)]

    # Write the updated DataFrame back to the CSV file
    #df.to_csv(file_path, index = False)
    # Reset the index of the DataFrame
    df_filt = df_filt.reset_index(drop = True)

    return df_filt

df1 = process_file(1,date)
df2 = process_file(2,date)
df3 = process_file(3,date)
#display(df1)
#display(df2)
#display(df3)

In [5]:
def plot_ble(df):
    # Plotly line graph
    fig = px.line(df, x='ts', y='rssi', color='id', title='Line Graph for Each ID')
    fig.update_layout(xaxis_title='Timestamp', yaxis_title='RSSI', legend_title='ID')
    fig.show()

plot_ble(df1)
plot_ble(df2)
plot_ble(df3)

c:\Users\isacc\miniconda3\Lib\site-packages\_plotly_utils\basevalidators.py:105: FutureWarning: The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result
  v = v.dt.to_pydatetime()


c:\Users\isacc\miniconda3\Lib\site-packages\_plotly_utils\basevalidators.py:105: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result



c:\Users\isacc\miniconda3\Lib\site-packages\_plotly_utils\basevalidators.py:105: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result



In [11]:
"""
2 inicio 17:57:30 -> 18:01:00
1 fim 18:02:00 -> 18:06:00
3 monta cargas 18:07:00 -> 18:10:00
"""
def av_ble(df1,df2,df3):
    # Set your desired time range
    #begginning -> ta no 2
    start_time1 = '2023-08-24 17:57:30'
    end_time1 = '2023-08-24 18:01:00'

    # Filter DataFrame based on the time range
    df1_1 = df1[(df1['ts'] >= start_time1) & (df1['ts'] <= end_time1)]
    #av_{station}_{time_interval}
    med1_1 = df1_1['rssi'].median()

    df2_1 = df2[(df2['ts'] >= start_time1) & (df2['ts'] <= end_time1)]
    med2_1 = df2_1['rssi'].median()

    df3_1 = df3[(df3['ts'] >= start_time1) & (df3['ts'] <= end_time1)]
    med3_1 = df3_1['rssi'].median()

    #####################################
    start_time2 = '2023-08-24 18:01:00'
    end_time2 = '2023-08-24 18:04:00'

    df1_2 = df1[(df1['ts'] >= start_time2) & (df1['ts'] <= end_time2)]
    med1_2 = df1_2['rssi'].median()

    df2_2 = df2[(df2['ts'] >= start_time2) & (df2['ts'] <= end_time2)]
    med2_2 = df2_2['rssi'].median()

    df3_2 = df3[(df3['ts'] >= start_time2) & (df3['ts'] <= end_time2)]
    med3_2 = df3_2['rssi'].median()


    start_time3 = '2023-08-24 18:07:00'
    end_time3 = '2023-08-24 18:10:00'

    df1_3 = df1[(df1['ts'] >= start_time3) & (df1['ts'] <= end_time3)]
    med1_3 = df1_3['rssi'].median()

    df2_3 = df2[(df2['ts'] >= start_time3) & (df2['ts'] <= end_time3)]
    med2_3 = df2_3['rssi'].median()

    df3_3 = df3[(df3['ts'] >= start_time3) & (df3['ts'] <= end_time3)]
    med3_3 = df3_3['rssi'].median()

    """
    2 inicio 17:57:30 -> 18:01:00
    1 fim 18:02:00 -> 18:06:00
    3 monta cargas 18:07:00 -> 18:10:00
    """

    station1_ref = {'receiver1': med1_2, 'receiver2': med2_2, 'receiver3': med3_2}
    station2_ref = {'receiver1': med1_1, 'receiver2': med2_1, 'receiver3': med3_1}
    station3_ref = {'receiver1': med1_3, 'receiver2': med2_3, 'receiver3': med3_3}
    print(station1_ref, station2_ref, station3_ref)

    return station1_ref, station2_ref, station3_ref

station1_ref, station2_ref, station3_ref = av_ble(df1,df2,df3)

def calculate_rms(df, ref_values):
    # Calculate RMS difference for each receiver
    rms_diff = np.sqrt(np.mean((df['rssi'] - np.array(list(ref_values.values())))**2))
    return rms_diff

def analyze_dataframes(df1, df2, df3, id_value):
    # Filter DataFrames for the given id value
    df1 = df1.loc[100:115]
    df2 = df2.loc[:15]
    df3 = df3.loc[80:100]
    #display(df1,df2,df3)
    df1_id = df1[df1['id'] == id_value]
    df2_id = df2[df2['id'] == id_value]
    df3_id = df3[df3['id'] == id_value]

    # Initialize variables for storing results
    #min_rms_df = None
    #min_rms_value = float('inf')

    # Iterate through timestamps in df1
    display(df1_id, df2_id, df3_id)
    for index, row in df1_id.iterrows():
        # Find the current timestamp in df1
        #print(row)
        print("ts")
        #ts_current = df1_id.loc[index, 'ts']
        ts_current = pd.to_datetime(row['ts'])
        print(ts_current)


        ts_end = ts_current + pd.Timedelta(seconds=30)
        df2_subset_next = df2_id[(df2_id['ts'] >= ts_current) & (df2_id['ts'] <= ts_end)]
        df3_subset_next = df3_id[(df3_id['ts'] >= ts_current) & (df3_id['ts'] <= ts_end)]

        if not df2_subset_next.empty:
            # Find the first occurrence of the next timestamp in df2 and df3
            ts_next_df2 = df2_subset_next.iloc[0]['ts']
            print(f'2 next: {ts_next_df2}')
        if not df3_subset_next.empty:
            ts_next_df3 = df3_subset_next.iloc[0]['ts']
            print(f'3 next: {ts_next_df3}')




        # Find the corresponding previous timestamp in df2 and df3
        df2_subset_prev = df2_id[df2_id['ts'] <= ts_current].tail(1)
        df3_subset_prev = df3_id[df3_id['ts'] <= ts_current].tail(1)

        # Find the first occurrence of the previous timestamp in df2 and df3
        if not df2_subset_prev.empty:
            ts_prev_df2 = df2_subset_prev.iloc[0]['ts']
            print(f'2 prev: {ts_prev_df2}')
        if not df3_subset_prev.empty:
            ts_prev_df3 = df3_subset_prev.iloc[0]['ts']
            print(f'3 prev: {ts_prev_df3}')
        print()
    
    return

        # Calculate RMS difference for each case
        #rms_df2_prev = calculate_rms(df2_id[df2_id['ts'] == ts_prev_df2], station2_ref)
        #rms_df3_prev = calculate_rms(df3_id[df3_id['ts'] == ts_prev_df3], station3_ref)

        #print("2")
        #display(df2_subset)
        #print("3")
        #display(df3_subset)
""""
        if not df2_subset.empty and not df3_subset.empty:
            # Calculate RMS difference for each case
            rms_df2 = calculate_rms(df2_subset, station2_ref)
            rms_df3 = calculate_rms(df3_subset, station3_ref)

            # Sum the RMS values
            rms_sum = rms_df2 + rms_df3"""

analyze_dataframes(df1, df2, df3, id_value=2)


{'receiver1': -55.0, 'receiver2': -68.0, 'receiver3': -63.0} {'receiver1': -68.0, 'receiver2': -61.0, 'receiver3': -71.0} {'receiver1': -66.0, 'receiver2': -71.5, 'receiver3': -61.0}


,ts,id,rssi
101,2023-08-24 17:50:50,2,-72
103,2023-08-24 17:50:56,2,-54
107,2023-08-24 17:51:14,2,-64
109,2023-08-24 17:51:44,2,-70
110,2023-08-24 17:51:50,2,-60


,ts,id,rssi
1,2023-08-24 17:49:07,2,-62
3,2023-08-24 17:49:19,2,-55
7,2023-08-24 17:50:13,2,-26
8,2023-08-24 17:50:19,2,-33
9,2023-08-24 17:50:42,2,-26
10,2023-08-24 17:51:01,2,-35
12,2023-08-24 17:51:17,2,-26
13,2023-08-24 17:56:45,2,-100
14,2023-08-24 17:56:50,2,-61
15,2023-08-24 17:57:02,2,-69


,ts,id,rssi
81,2023-08-24 17:46:05,2,-79
83,2023-08-24 17:46:11,2,-66
85,2023-08-24 17:46:23,2,-73
86,2023-08-24 17:46:41,2,-66
87,2023-08-24 17:46:47,2,-72
88,2023-08-24 17:47:29,2,-79
89,2023-08-24 17:47:41,2,-73
93,2023-08-24 17:57:59,2,-76
94,2023-08-24 17:58:11,2,-70
95,2023-08-24 17:58:17,2,-78


ts
2023-08-24 17:50:50
2 next: 2023-08-24 17:51:01
2 prev: 2023-08-24 17:50:42
3 prev: 2023-08-24 17:47:41

ts
2023-08-24 17:50:56
2 next: 2023-08-24 17:51:01
2 prev: 2023-08-24 17:50:42
3 prev: 2023-08-24 17:47:41

ts
2023-08-24 17:51:14
2 next: 2023-08-24 17:51:17
2 prev: 2023-08-24 17:51:01
3 prev: 2023-08-24 17:47:41

ts
2023-08-24 17:51:44
2 prev: 2023-08-24 17:51:17
3 prev: 2023-08-24 17:47:41

ts
2023-08-24 17:51:50
2 prev: 2023-08-24 17:51:17
3 prev: 2023-08-24 17:47:41

